### Basic library imports

In [1]:
import os
import pandas as pd

### Read Dataset

In [2]:
DATASET_FOLDER = '../dataset/'
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))

### Run Sanity check using src/sanity.py

In [ ]:
!python sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out.csv

In [ ]:
!python sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out_fail.csv

### Download images

In [ ]:
from utils import download_images
download_images(sample_test['image_link'], '../images')

In [ ]:
from utils import process_images_with_easyocr

# Process images and extract quantitative data
ocr_results = process_images_with_easyocr('../images')

# Print extracted quantitative data
for image_name, data in ocr_results.items():
    print(f"Quantitative data extracted from {image_name}: {data}")

In [ ]:
# Shivansh pytesseract
from utils import process_images_with_ocr

ocr_results = process_images_with_ocr(sample_test['image_link'], '../del_images')

for image_name, text in ocr_results.items():
    print(f"Text extracted from {image_name}: {text}")

In [ ]:
assert len(os.listdir('../images')) > 0

In [15]:
rm -rf ../images

In [ ]:
import easyocr
import re

# Load EasyOCR model
reader = easyocr.Reader(['en'])

# Define allowed units based on entity type (you can expand this)
units_dict = {
    'width': ['millimetre', 'centimetre', 'metre', 'inch', 'foot', 'yard'],
    'depth': ['millimetre', 'centimetre', 'metre', 'inch', 'foot', 'yard'],
    'height': ['millimetre', 'centimetre', 'metre', 'inch', 'foot', 'yard'],
    'voltage': ['volt', 'kilovolt', 'millivolt'],
    'wattage': ['watt', 'kilowatt'],
    'item_weight': ['gram', 'kilogram', 'ton', 'pound'],
    'maximum_weight_recommendation': ['gram', 'kilogram', 'ton', 'pound']
}

# Function to extract the prediction from OCR and process it
def predictor(image_link, entity_name):
    # Perform OCR to extract text from the image
    ocr_result = reader.readtext(image_link, detail=0)
    
    # Join OCR results into a single string for easier searching
    ocr_text = " ".join(ocr_result).lower()
    
    # Get allowed units for the given entity
    allowed_units = units_dict.get(entity_name, [])
    
    # Regex pattern to match value followed by unit (for any of the allowed units)
    pattern = r'(\d+\.?\d*)\s?(' + '|'.join(allowed_units) + ')'
    
    # Search for the pattern in the OCR text
    match = re.search(pattern, ocr_text)
    
    if match:
        value = match.group(1)  # Extract the value
        unit = match.group(2)   # Extract the unit
        return f"{value} {unit}"
    
    # Return empty string if no match is found
    return ""

# Sample usage
image_link = "https://m.media-amazon.com/images/I/41-NCxNuBxL.jpg"
entity_name = "width"
prediction = predictor(image_link, entity_name)
print(prediction)
predictor("https://m.media-amazon.com/images/I/41-NCxNuBxL.jpg", "width")

ocr_result = reader.readtext(image_link, detail=0)
print("OCR Output:", ocr_result)
import cv2
from PIL import Image
import numpy as np

# Download image
image_path = '../images/41-NCxNuBxL.jpg'
image = Image.open(image_path)

# Convert to grayscale
gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)

# Apply thresholding
_, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

# Save the processed image (optional)
cv2.imwrite("processed_image.jpg", thresh)

# Pass the processed image to EasyOCR
ocr_result = reader.readtext(thresh, detail=0)
print("OCR Output:", ocr_result)


### Working Code (Not for all but for more than 1)

In [ ]:
import easyocr
import re
import cv2
from PIL import Image
import numpy as np
import os

# Load EasyOCR model
reader = easyocr.Reader(['en'])

# Updated entity-unit map
entity_unit_map = {
    "width": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "depth": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "height": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
    "item_weight": {"milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"},
    "maximum_weight_recommendation": {"milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound"},
    "voltage": {"millivolt", "kilovolt", "volt"},
    "wattage": {"kilowatt", "watt"},
    "item_volume": {"cubic foot", "microlitre", "cup", "fluid ounce", "centilitre", "imperial gallon", "pint", "decilitre", "litre", "millilitre", "quart", "cubic inch", "gallon"}
}

# Function to extract the prediction from OCR and process it
def predictor(image_path):
    # Open the image and apply preprocessing
    image = Image.open(image_path)
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    # Perform OCR to extract text from the processed image
    ocr_result = reader.readtext(thresh, detail=0)
    
    # Join OCR results into a single string for easier searching
    ocr_text = " ".join(ocr_result).lower()

    # Results to store predictions
    results = []

    # Loop through the entity_unit_map and try to find any matches
    for entity_name, allowed_units in entity_unit_map.items():
        # Build regex pattern for matching the entity and allowed units
        pattern = r'(\d+\.?\d*)\s?(' + '|'.join(allowed_units) + ')'
        
        # Search for the pattern in the OCR text
        match = re.search(pattern, ocr_text)
        
        if match:
            value = match.group(1)  # Extract the value
            unit = match.group(2)   # Extract the unit
            results.append(f"{entity_name}: {value} {unit}")
    
    # If results are found, return them, otherwise return empty string
    if results:
        return " ".join(results)
    else:
        return ""


# Automatically get all image paths from the 'images' folder
image_folder = '../images'  # Replace with your folder path
image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) if fname.endswith(('.jpg', '.png', '.jpeg'))]

# Process all images and print results
for idx, image_path in enumerate(image_paths, 1):
    result = predictor(image_path)
    if result:
        print(f"{idx}. {result}")
    else:
        print(f"{idx}. No entities detected")
